# Chaining and Retrieval

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("educate.pdf")
docs = loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embeddings)

C:\Users\dell\AppData\Local\Temp\ipykernel_16388\4051320023.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
d:\AIML\LangChain-Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_community.llms import Ollama
# load llama2 model
llm=Ollama(model="llama2")

C:\Users\dell\AppData\Local\Temp\ipykernel_16388\4179488048.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="llama2")


In [7]:
# Design prompt template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the context provided below.
Think step by step and provide a detailed answer.
<context>
{context}
</context>
Question: {input}""")

In [8]:
# chain introduction
# Create document chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [9]:
"""Retrivers: An interface which connects to the vector db and returns documents given an unstructured query
It does not need to be able to store the documents, just return them."""

retriver = db.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B429575D50>, search_kwargs={})

In [10]:
# Retriever Chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriver, document_chain)

In [15]:
retrieval_chain.invoke({"input" : "What is an activation function? Explain in simple terms"})['answer']

'In simple terms, an activation function is a mathematical function that takes a input value and transforms it into an output value with a specific shape or pattern. The output value can be either a number (in the case of a continuous activation function) or a boolean value (in the case of a discrete activation function).\n\nThe main purpose of an activation function is to introduce non-linearity into a neural network, which allows it to learn and represent more complex relationships between inputs and outputs. Without activation functions, neural networks would only be able to learn linear relationships, which are not sufficient for solving many real-world problems.\n\nSome common activation functions used in deep learning include:\n\n1. Sigmoid: This function maps the input value to a output value between 0 and 1. It is often used in the hidden layers of a neural network, as it allows the network to learn linear or logistic relationships between the inputs and outputs.\n2. ReLU (Rect